In [ ]:
#|default_exp search

In [ ]:
#|export
import json
import os

import click
import numpy as np
import pandas as pd
import requests

In [ ]:
#|export
try:
    from dotenv import load_dotenv

    load_dotenv()
    key = os.getenv("ELSEVIER_API_KEY")
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "x-els-apikey": key,
    }
except KeyError:
    raise Exception("Please set the ELSEVIER_API_KEY environment variable")

In [ ]:
#|hide
#|export
def _fitler_null_authors(authors):
    if authors is np.nan:
        return 'Author not available'
    return ', '.join([i['name'] for i in authors])

In [ ]:
#|hide
#|export
def _process_response(response):
    if response.status_code == 200:
        print("Total results: ", response.json()["resultsFound"])

        json_data = json.loads(response.text)['results']

        df = pd.json_normalize(json_data)

        df['authors'] = df['authors'].apply(_fitler_null_authors)

        return df
    elif response.status_code == 404:
        print("No results found")
        return None



In [ ]:
#|export
@click.command()
@click.option("--query", "-q", help="Query string")
@click.option("--publication", "-p", help="Publication name")
@click.option("--open_access", "-oa", help="Open access", type=bool, default=False)
@click.option("--offset", "-o", help="Offset", type=int, default=0)
@click.option("--count", "-c", help="Number of results", type=int,default=25)
@click.option("--sort_by", "-sb", help="Sort by", type=str, default="date")
@click.option("--save_to_csv", "-csv", help="Save to csv", type=str, default="results")
def search(query, publication, open_access, offset, count, sort_by, save_to_csv):

    json_data = {
        "qs": query,
        "pub": publication,
        "filters": {
            "openAccess": open_access,
        },
        "display": {
            "offset": offset,
            "show": count,
            "sortBy": sort_by ,
        },
    }


    response = requests.put(
        "https://api.elsevier.com/content/search/sciencedirect",
        headers=headers,
        json=json_data,
    )
    response = _process_response(response)
    if response is not None:
        name = f"{query=} {publication=} {open_access=} {offset=} {count=} {sort_by=}"
        response.to_csv(f"{name}.csv", index=False)
        print(f"Results saved to {name}.csv")
